In [6]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import gc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# прописывем нужные нам пути
dir_project = 'proj_news_radar/proj_news_viz/nlp' # путь проекта

PATH = os.path.join(os.getenv('HOME'), dir_project)
sys.path.append(PATH)

In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import multiprocessing as mp
from multiprocessing import Pool
num_cores = int(mp.cpu_count() / 2) # number of cores on your machine; 20
num_partitions = num_cores * 2        # number of partitions to split dataframe

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

---
возьмем для примера датасет [lenta.ru](https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta)

для примера взят датасет, скаченный по линку с gitHub

In [15]:
df = pd.read_csv(f'{PATH}/data/external/news_lenta.csv')
print(df.shape)
df.head(2)

(699746, 5)


,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,https://lenta.ru/news/2018/07/16/foes/


In [16]:
# удалим пустые записи и дубликаты по признаку `text`
df.dropna(inplace=True)
df.text.drop_duplicates(inplace=True)

print(f'df.shape: {df.shape}')
df.isnull().sum()

df.shape: (675697, 5)


tags     0
text     0
title    0
topic    0
url      0
dtype: int64

In [26]:
# конкатенируем признаки `title` & `text`
df['full_text'] = df.title + ' ' + df.text
df.sample(2)

,tags,text,title,topic,url,full_text
555150,Все,Российские военнослужащих в ближайшее время бу...,Российские миротворцы отправятся в Судан,Россия,https://lenta.ru/news/2006/02/10/secretary/,Российские миротворцы отправятся в Судан Росси...
32368,Общество,"На несогласованных акциях, проходящих 7 октябр...",На несогласованных акциях в российских городах...,Россия,https://lenta.ru/news/2017/10/07/kadirov/,На несогласованных акциях в российских городах...


---
препроцессим

In [11]:
from news_viz.data_processing.preprocessing_tools import clean_text, lemmatization

* clean text

In [27]:
def multiply_clean_text(data):
    data['clean_text'] = data['text'].apply(clean_text)
    return data

In [28]:
%%time

df = parallelize_dataframe(df, multiply_clean_text)
df = df[df.clean_text!='9999']

gc.collect()

CPU times: user 17 s, sys: 26.7 s, total: 43.8 s
Wall time: 1min 23s


61